In [1]:
import torch
import numpy as np
import pandas as pd
from src.preprocessing import remove_stop_words
from src.model_architectures.bert_classifier import SiameseNetwork

In [2]:
net = SiameseNetwork(50)

In [4]:
for idx, module in enumerate(net.bert.modules()):
    print(type(module))
    if type(module) == torch.nn.modules.dropout.Dropout:
        module = torch.nn.Dropout(p=0.3)
    print(idx, module)

<class 'transformers.modeling_bert.BertModel'>
0 BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (d

In [5]:
for idx, module in enumerate(net.bert.modules()):
    print(idx, module)

0 BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
        

In [112]:
df = pd.read_csv('data/train/spec_train_data_new.csv')

In [192]:
label_0 = [0, 0, 0]
label_1 = [0, 0, 0]
token = '256 gb'
for row in df.iloc:
    if token in row.title_one and token in row.title_two:
        if row.label == 1:
            label_1[0] += 1
        else:
            label_0[0] += 1
    
    elif token in row.title_one and token not in row.title_two:
        if row.label == 1:
            label_1[1] += 1
        else:
            label_0[1] += 1
    
    elif token not in row.title_one and token in row.title_two:
        if row.label == 1:
            label_1[2] += 1
        else:
            label_0[2] += 1

print("In both titles: ", label_0[0], label_1[0])
print("In title one, not title 2: ", label_0[1], label_1[1])
print("In title two not title 1: ", label_0[2], label_1[2])

In both titles:  370 481
In title one, not title 2:  111 0
In title two not title 1:  125 0


In [241]:
net = torch.load('./models/0.2.1.1_BERT_epoch_5.pt')
net.eval()
print('Loaded Model')

Loaded Model


In [178]:
#torch.save(net.state_dict(), './models/0.2.1.1_BERT_epoch_1_state_dict.pt')

In [180]:
def inference():
    title1 = input('First title: ')
    title2 = input('Second title: ')
    
    title1 = remove_stop_words(title1)
    title2 = remove_stop_words(title2)
    
    data = np.array([title1, title2]).reshape(1, 2)
    forward = net(data)
    np_forward = forward.detach().numpy()[0]
    
    print('Output: {}'.format(torch.argmax(forward)))
    print('Softmax: Negative {:.4f}%, Positive {:.4f}%'.format(np_forward[0], np_forward[1]))

In [252]:
inference()

First title: ASUS ZenBook 14 Ultra-Slim Laptop 14" Full HD NanoEdge Bezel, Intel Core i7-1065G7, 8 GB RAM, 32 GB PCIe SSD, NumberPad, Thunderbolt 3, Windows 10 Home, Pine Grey, UX425JA-EB71
Second title: ASUS - ZenBook 14" Ultra-Slim FHD Notebook - i7-1065G7 - 8 GB 32 GB in Pine Grey - Pine Grey
Output: 1
Softmax: Negative 0.0001%, Positive 0.9999%


In [269]:
inference()

First title: Dell XPS 13 7390 Laptop 13.3 inch, 4K UHD InfinityEdge Touch, 10th Gen Intel Core i7, Intel UHD Graphics, 256 gb SSD, 16GB RAM, Windows 10 Home, XPS7390-7681SLV-PUS
Second title: Dell XPS 13 7390 Laptop 13.3 inch, 4K UHD InfinityEdge Touch, 10th Gen Intel Core i7, Intel UHD Graphics, 512 gb SSD, 16GB RAM, Windows 10 Home, XPS7390-7681SLV-PUS
Output: 1
Softmax: Negative 0.0001%, Positive 0.9999%


In [236]:
inference()

First title: apple macbook 15 4"" laptop 496 gb sata ssd 4 gb ram amd ryzen 5 1600 3 2 ghz 6 core cpu
Second title: apple macbook 15 4"" laptop 256 gb sata ssd 4 gb ram amd ryzen 5 1600 3 2 ghz 6 core cpu
Output: 0
Softmax: Negative 0.9999%, Positive 0.0001%


In [203]:
inference()

First title: ASUS VivoBook Thin and Lightweight FHD WideView Laptop, 8th Gen Intel Core i5-8250U, 8GB DDR4 RAM, 128GB SSD+1TB HDD, USB Type-C, NanoEdge, Fingerprint Reader, Windows 10 - F510UA-AH55
Second title: ASUS Laptop 15.6, Intel Core i5-8250U 1.6GHz, Intel HD, 1TB HDD + 256 GB SSD, 16 GB RAM, F510UA-AH55
Output: 1
Softmax: Negative 0.0009%, Positive 0.9991%


In [268]:
inference()

First title: 1 tb
Second title: 2 tb
Output: 1
Softmax: Negative 0.0015%, Positive 0.9985%


In [ ]:
hp 15 4 inch laptop intel core i7 4770k 3 5 ghz 480 gb hard drive 104 gb memory

hp chromebook 15 4"" intel core i7 4770k cpu 480 gb hdd 104 gb ram

In [ ]:
16 gb ram 512 gb ssd asus zenbook ryzen cpu